# Create Aurora Borealis Art using GANs and PyTorch

Based on https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/pytorch/README.rst and https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

In [1]:
from sagemaker import get_execution_role
role = get_execution_role()
print(role)

arn:aws:iam::019326146125:role/service-role/AmazonSageMaker-ExecutionRole-20190407T165524


In [9]:
from sagemaker.pytorch import PyTorch

pytorch_estimator = PyTorch(entry_point='../train-sagemaker.py',
                            role=role,
                            train_instance_type='ml.p2.xlarge',  # 'ml.p3.2xlarge',
                            train_instance_count=1,
                            framework_version='1.0.0',
                            output_path='s3://markstrefford-art-1/aurora-model-1',
                            # output_path='s3://markstrefford-art-1/facegen-gan/model',
                            hyperparameters = {'epochs': 1})

* Now set up training a model

In [10]:
pytorch_estimator.fit({'training': 's3://markstrefford-art-1/aurora/center-aligned'})
# pytorch_estimator.fit({'training': 's3://markstrefford-art-1/celebA/Img/img_align_celeba'})



INFO:sagemaker:Creating training-job with name: sagemaker-pytorch-2019-04-25-15-36-17-740


2019-04-25 15:36:18 Starting - Starting the training job...
2019-04-25 15:36:19 Starting - Launching requested ML instances.........
2019-04-25 15:37:57 Starting - Preparing the instances for training......
2019-04-25 15:39:07 Downloading - Downloading input data...
2019-04-25 15:39:22 Training - Downloading the training image...
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-04-25 15:40:07,555 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-04-25 15:40:07,582 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-04-25 15:40:08,188 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-04-25 15:40:08,469 sagemaker-containers INFO     Module train-sagemaker does not provide a setup.py. 
Generating setup.py
2019-04-25 15:40:08,469 sagemaker-containers INFO     Generating setup.cfg
2019-04-25 15:40:08,469 sagemak


2019-04-25 15:40:35 Training - Training image download completed. Training in progress.[0/1][0/5]#011Loss_D: 1.7398#011Loss_G: 5.2860#011D(x): 0.5259#011D(G(z)): 0.5669 / 0.0072
Saving final model: {'epoch': 0, 'lr': 0.0002, 'errD': 1.7397509813308716, 'errG': 5.285957336425781, 'D_x': 0.5259150266647339, 'D_G_z1': 0.5669409036636353, 'D_G_z2': 0.007243373431265354}
2019-04-25 15:41:47,777 sagemaker-containers INFO     Reporting training SUCCESS

2019-04-25 15:42:03 Uploading - Uploading generated training model
2019-04-25 15:42:03 Completed - Training job completed
Billable seconds: 177
